<a href="https://colab.research.google.com/github/pedroescobedob/DS-Unit-2-Linear-Models/blob/master/Pedro_Escobedo_assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [58]:
df.head()

bathrooms  bedrooms  ... wheelchair_access common_outdoor_space
0        1.5         3  ...                 0                    0
1        1.0         2  ...                 0                    0
2        1.0         1  ...                 0                    0
3        1.0         1  ...                 0                    0
4        1.0         4  ...                 0                    0

[5 rows x 34 columns]

In [0]:
# Change the date.
df['created'] = pd.to_datetime(df['created'])

In [0]:
trainDf = df[(df['created'] >= pd.to_datetime('April 1st, 2016')) & (df['created'] < pd.to_datetime('June 1st, 2016'))]
testDf = df[(df['created'] >= pd.to_datetime('April 1st, 2016')) & (df['created'] < pd.to_datetime('July 1st, 2016'))]

In [0]:
# Two new features
df['description length'] = df['description'].str.len().fillna(0)
df['total perks'] = df[df.columns[10:]].T.sum()

In [0]:
# Make interest into a measurable quantity.
int_values = {'low': 0, 'medium': 1, 'high': 2}
df['interest_level'] = df['interest_level'].map(int_values)

In [63]:
df.head()

bathrooms  bedrooms  ... description length total perks
0        1.5         3  ...              588.0       588.0
1        1.0         2  ...                8.0        13.0
2        1.0         1  ...              691.0       694.0
3        1.0         1  ...              492.0       494.0
4        1.0         4  ...              479.0       480.0

[5 rows x 36 columns]

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [0]:
features = ['description length', 'total perks', 'bathrooms', 'bedrooms']
# df.reindex(columns=['description length', 'total perks'])
target = ['price']
model = LinearRegression()

In [0]:
# Train
X_train = trainDf[features]
y_train = trainDf[target]

In [90]:
model.fit(X_train, y_train)
model.intercept_, model.coef_

(array([279.62466944]),
 array([[ -78.44198228,   78.43619611, 1936.34568335,  392.08336268]]))

In [0]:
# Predictions
y_pred = model.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)

In [93]:
print('Intercept', model.intercept_)
# coefficients = pd.Series(model.coef_, features)
# print(coefficients.to_string())
print('r2: ', model.score(X_train, y_train))

print('MSE: ', mse)
print('RMSE: ', rmse)
print('MAE: ', mae)
print('r2: ', r2)

Intercept [279.62466944]
r2:  0.533180960768856
MSE:  1449486.2894615808
RMSE:  1203.9461322922969
MAE:  795.3864825480687
r2:  0.533180960768856


In [94]:
# Test

X_test = testDf[features]
y_test = testDf[target]

model.fit(X_test, y_test)
model.intercept_, model.coef_

(array([264.96148191]),
 array([[ -79.0865947 ,   79.07489766, 1960.13672423,  388.05400726]]))

In [0]:
y_pred = model.predict(X_test)

# regression metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [97]:
print('Intercept', model.intercept_)
# coefficients = pd.Series(model.coef_, features)
# print(coefficients.to_string())
print('r2: ', model.score(X_test, y_test))

print('MSE: ', mse)
print('RMSE: ', rmse)
print('MAE: ', mae)
print('r2: ', r2)

Intercept [264.96148191]
r2:  0.5365751702905422
MSE:  1439443.2137528218
RMSE:  1199.767983300447
MAE:  798.3405213818272
r2:  0.5365751702905422
